# CatBoost Starter for Brain Comp
EEGの差分の集計特徴量を用いたNotebook。

- ver1: CV , LB 

### Version Notes
- Version 1 - EEGのT=20, 30, 40秒における、10秒時間窓 / 20秒時間窓のmean, min, max, std, max-minを特徴量とした

# Load Libraries

In [ ]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"
import pandas as pd
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
pl.Config.set_tbl_cols(-1)

VER = 2

# Load Train Data

In [ ]:
df_train = pl.read_csv('../input/hms-harmful-brain-activity-classification/train.csv')
TARGETS = df_train.columns[-6:]
print('Train shape:', df_train.shape )
print('Targets', list(TARGETS))
df_train.head()

# Create Multiple Eeg Id Train Data
データの説明[here][1]には、テストデータには、同じ `eeg_id` からの複数の crop は含まれていないと記載されている。<br>
ひとつの`eeg_id`ごとのデータにする

[1]: https://www.kaggle.com/competitions/hms-harmful-brain-activity-classification/discussion/467021

In [ ]:
df_train = df_train.group_by(by = 'eeg_id').agg(
    pl.min("eeg_label_offset_seconds").alias("eeg_min_offset"),
    pl.max("eeg_label_offset_seconds").alias("eeg_max_offset"),
    pl.first("patient_id"),
    pl.sum("seizure_vote"),
    pl.sum("lpd_vote"),
    pl.sum("gpd_vote"),
    pl.sum("lrda_vote"),
    pl.sum("grda_vote"),
    pl.sum("other_vote"),
    pl.first("expert_consensus")
)
df_train.head()


# 各vote列の合計
df_train = df_train.with_columns(
    (pl.col('seizure_vote') + pl.col('lpd_vote') + pl.col('gpd_vote') + pl.col('lrda_vote') + pl.col('grda_vote') + pl.col('other_vote')).alias('vote_sum')
)

# 各vote列の正規化
df_train = df_train.with_columns(
    (pl.col('seizure_vote') / pl.col('vote_sum')).alias('seizure_vote'),
    (pl.col('lpd_vote') / pl.col('vote_sum')).alias('lpd_vote'),
    (pl.col('gpd_vote') / pl.col('vote_sum')).alias('gpd_vote'),
    (pl.col('lrda_vote') / pl.col('vote_sum')).alias('lrda_vote'),
    (pl.col('grda_vote') / pl.col('vote_sum')).alias('grda_vote'),
    (pl.col('other_vote') / pl.col('vote_sum')).alias('other_vote'),
)

# vote列の削除
df_train = df_train.drop(["vote_sum"])

# expert_consensus列をtarget列とする
df_train = df_train.rename({"expert_consensus": "target"})
df_train.head()

# Feature Engineer


CatBoostモデルに入れる特徴量を作成する。


In [ ]:
# parameter
# 差分特徴量の集計特徴量をつくる
# window = 4000 # 20秒窓 (20秒 / (1レコード1/200秒) = 4000レコード)
window_list = [1000, 2000]
center_seconds_list = [20, 30]

In [ ]:
def create_aggregated_features_on_EEG_differences(df_eeg_diff, window, center_sec):
    """
    @Args
    df_eeg_diff    : 差分特徴量を持つデータフレーム
    window      : 時間窓として抽出する行数 [行]
    center_sec  : 時間窓の中心時刻 [s]
    
    @Returns
    df_eeg_diff    : 集計特徴量を持つデータフレーム
    
    """
    window_time = window / (200)
    str_window = str(int(window_time)) + "s"

    # 時間窓 [window_min, window_max] に入るレコードのみ抽出
    window_max = center_sec + (window / 2)
    window_min = center_sec - (window / 2)

    df_eeg_diff = df_eeg_diff.filter((pl.col("seconds_eeg") <= window_max) & (pl.col("seconds_eeg") >= window_min))
    df_eeg_diff = df_eeg_diff.drop(['seconds_eeg'])
    
    df_eeg_diff = df_eeg_diff.with_columns(
        # LL: Fp1 - T3
        pl.col("LL_Fp1-T3").mean().alias("mean_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").max().alias("min_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").min().alias("max_" + str_window + "_LL_Fp1-T3"),
        pl.col("LL_Fp1-T3").std().alias("std_" + str_window + "_LL_Fp1-T3"),
        (pl.col("LL_Fp1-T3").max() - pl.col("LL_Fp1-T3").min()).alias("max-min_" + str_window + "_LL_Fp1-T3"),

        # LL: Fp1 - T3
        pl.col("LL_T3-O1").mean().alias("mean_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").max().alias("min_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").min().alias("max_" + str_window + "_LL_T3-O1"),
        pl.col("LL_T3-O1").std().alias("std_" + str_window + "_LL_T3-O1"),
        (pl.col("LL_T3-O1").max() - pl.col("LL_T3-O1").min()).alias("max-min__" + str_window + "_LL_T3-O1"),


        # LP: Fp1 - C3
        pl.col("LP_Fp1-C3").mean().alias("mean_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").max().alias("min_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").min().alias("max_" + str_window + "_LP_Fp1-C3"),
        pl.col("LP_Fp1-C3").std().alias("std_" + str_window + "_LP_Fp1-C3"),
        (pl.col("LP_Fp1-C3").max() - pl.col("LP_Fp1-C3").min()).alias("max-min_" + str_window + "_LP_Fp1-C3"),

        # LP: C3 - O1
        pl.col("LP_C3-O1").mean().alias("mean_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").max().alias("min_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").min().alias("max_" + str_window + "_LP_C3-O1"),
        pl.col("LP_C3-O1").std().alias("std_" + str_window + "_LP_C3-O1"),
        (pl.col("LP_C3-O1").max() - pl.col("LP_C3-O1").min()).alias("max-min_" + str_window + "_LP_C3-O1"),


        # RP: Fp1 - C4
        pl.col("RP_Fp2-C4").mean().alias("mean_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").max().alias("min_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").min().alias("max_" + str_window + "_RP_Fp2-C4"),
        pl.col("RP_Fp2-C4").std().alias("std_" + str_window + "_RP_Fp2-C4"),
        (pl.col("RP_Fp2-C4").max() - pl.col("RP_Fp2-C4").min()).alias("max-min_" + str_window + "_RP_Fp2-C4"),

        # RP: C4 - O2
        pl.col("RP_C4-O2").mean().alias("mean_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").max().alias("min_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").min().alias("max_" + str_window + "_RP_C4-O2"),
        pl.col("RP_C4-O2").std().alias("std_" + str_window + "_RP_C4-O2"),
        (pl.col("RP_C4-O2").max() - pl.col("RP_C4-O2").min()).alias("max-min_" + str_window + "_RP_C4-O2"),


        # RR: Fp2 - T4
        pl.col("RR_Fp2-T4").mean().alias("mean_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").max().alias("min_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").min().alias("max_" + str_window + "_RR_Fp2-T4"),
        pl.col("RR_Fp2-T4").std().alias("std_" + str_window + "_RR_Fp2-T4"),
        (pl.col("RR_Fp2-T4").max() - pl.col("RR_Fp2-T4").min()).alias("max-min_" + str_window + "_RR_Fp2-T4"),

        # RR: T4 - O2
        pl.col("RR_T4-O2").mean().alias("mean_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").max().alias("min_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").min().alias("max_" + str_window + "_RR_T4-O2"),
        pl.col("RR_T4-O2").std().alias("std_" + str_window + "_RR_T4-O2"),
        (pl.col("RR_T4-O2").max() - pl.col("RR_T4-O2").min()).alias("max-min_" + str_window + "_RR_T4-O2"),
    )

    
    # 差分特徴量をdrop
    df_eeg_diff = df_eeg_diff.drop(['LL_Fp1-T3', 'LL_T3-O1', 'LP_Fp1-C3', 'LP_C3-O1', 'RP_Fp2-C4', 'RP_C4-O2', 'RR_Fp2-T4', 'RR_T4-O2'])
    
    # 列名を一括で変更
    col_suffix = "_at_" + str(center_sec) + "s"
    list_original_col = df_eeg_diff.columns
    df_eeg_diff = df_eeg_diff.with_columns(pl.all().name.suffix(col_suffix)).drop(list_original_col) # suffixを付与して、元の列名の列を削除

    # print(df_eeg_diff.head())

    return df_eeg_diff

In [ ]:
%%time
PATH = '../input/hms-harmful-brain-activity-classification/train_eegs/'
files = os.listdir(PATH)
print(f'There are {len(files)} eeg parquet files')

all_eegs = {}
temp = pl.DataFrame()
for i, f in enumerate(files):
    if i%100 == 0: print(i, ', ', end = '')

    # if i%10 != 0: continue

    # 990852758
    eeg_id = int(f.split('.')[0])    

    temp_eeg = pl.read_parquet(f'{PATH}{f}')

    # eegデータにおける、計測開始から各行までの秒数 seconds_eeg
    t = 1/200 # EEGのサンプリング周波数は200Hzなので、1行は1/200秒ごとの計測値を表す
    temp_eeg = temp_eeg.with_row_count("id")
    temp_eeg = temp_eeg.with_columns((pl.col("id") * t).alias("seconds_eeg"))   # 計測時刻の列をつくる
    temp_eeg = temp_eeg.filter(pl.col("seconds_eeg") < 50)                      # 50秒以降のデータは使わない
    temp_eeg = temp_eeg.drop(["id"])
    
    # 差分特徴量を作る
    temp_eeg = temp_eeg.with_columns(
        (pl.col("Fp1") - pl.col("T3")).alias("LL_Fp1-T3"),
        (pl.col("T3") - pl.col("O1")).alias("LL_T3-O1"),
        (pl.col("Fp1") - pl.col("C3")).alias("LP_Fp1-C3"),
        (pl.col("C3") - pl.col("O1")).alias("LP_C3-O1"),
        (pl.col("Fp2") - pl.col("C4")).alias("RP_Fp2-C4"),
        (pl.col("C4") - pl.col("O2")).alias("RP_C4-O2"),
        (pl.col("Fp2") - pl.col("T4")).alias("RR_Fp2-T4"),
        (pl.col("T4") - pl.col("O2")).alias("RR_T4-O2"),
    )


    # 差分特徴量以外drop
    temp_eeg = temp_eeg.drop(['Fp1', 'F3', 'C3', 'P3', 'F7', 'T3', 'T5', 'O1', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'C4', 'P4', 'F8', 'T4', 'T6', 'O2', 'EKG'])
    

    df_eeg_feature = pl.DataFrame()
    for center_sec in center_seconds_list:
        for window in window_list:
            # print(center_sec)
            # print(window)
            temp_eeg_sec_window = create_aggregated_features_on_EEG_differences(temp_eeg, window, center_sec)
            df_eeg_feature = pl.concat([df_eeg_feature, temp_eeg_sec_window], how = 'horizontal')

    # eeg_id を追加
    df_eeg_feature = df_eeg_feature.with_columns(pl.lit(str(eeg_id)).alias("eeg_id"))

    # 全ての eeg_id に対して結合
    temp = pl.concat([temp, df_eeg_feature], how = 'vertical')

In [ ]:
temp.head()

In [ ]:
temp = temp.with_columns(pl.col("eeg_id").cast(pl.Int64).alias("eeg_id"))
temp.tail(5)

In [ ]:
# join
df_train = df_train.join(temp, on = "eeg_id", how = "left")
df_train.head()

In [ ]:
# Testデータでは、eeg_id, patient_id, spectrogram_id, label_idは1つしかないため、trainingにも使わない
# patient_idについて、データの分割に利用
FEATURES = df_train.drop(["eeg_id", "eeg_sub_id", "eeg_label_offset_seconds", "eeg_max_offset", "eeg_min_offset",
                           "spectrogram_id", "spectrogram_sub_id", "spectrogram_label_offset_seconds",
                           "patient_id", "label_id", "seconds_eeg",
                           "seizure_vote", "lpd_vote", "gpd_vote", "lrda_vote", "grda_vote", "other_vote", "vote_sum",
                           "EKG", "target",
                           ]).columns
FEATURES

In [ ]:
# pandasに変換
df_train = df_train.to_pandas()

In [ ]:
# Target変数のラベルの定義
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
df_train['target'] = df_train['target'].replace(TARS)

# Train CatBoost
CatBoost は、超高速トレーニングのために (パラメーター `task_type='GPU'` を追加すると) 両方の Kaggle T4 GPU を自動的に使用する。

In [ ]:
import catboost as cat
from catboost import CatBoostClassifier, Pool
print('CatBoost version',cat.__version__)

In [ ]:
if not os.path.exists('models'):
    os.makedirs('models')

In [ ]:
%%time
from sklearn.model_selection import KFold, GroupKFold
import json

all_oof = []
all_true = []

gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(df_train, df_train.target, df_train.patient_id)):   
    
    print('#'*25)
    print(f'### Fold {i+1}')
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)

    model = CatBoostClassifier(loss_function = 'MultiClass')    
    # model = CatBoostClassifier(task_type='GPU',
    #                            loss_function='MultiClass')
    
    # trainデータ / validデータのPoolを作成
    train_pool = Pool(
        data = df_train.loc[train_index,FEATURES],
        label = df_train.loc[train_index,'target'],
    )
    
    valid_pool = Pool(
        data = df_train.loc[valid_index,FEATURES],
        label = df_train.loc[valid_index,'target'],
    )
    
    model.fit(train_pool,
             verbose=100,
             eval_set=valid_pool,
             )
    
    # モデルの保存
    model.save_model(f'models/CAT_v{VER}_f{i}.cat')

    # モデルのパラメータの保存
    params = model.get_all_params()
    with open(f'models/CAT_v{VER}_f{i}_params.txt', 'w') as f:
        json.dump(params, f)

    # 各 fold の予測確率
    oof = model.predict_proba(valid_pool)
    all_oof.append(oof)
    all_true.append(df_train.loc[valid_index, TARGETS].values)
    
    del train_pool, valid_pool, oof #model
    gc.collect()
    
    #break
    
all_oof = np.concatenate(all_oof)   # 全validationデータに対する予測確率
all_true = np.concatenate(all_true) # 全validationデータのターゲット変数の実測値

# Feature Importance
Below we display the CatBoost top 25 feature importance for the last fold we trained.

feature importance 上位25件

In [ ]:
TOP = 25

feature_importance = model.feature_importances_
sorted_idx = np.argsort(feature_importance)
fig = plt.figure(figsize=(10, 8))
plt.barh(np.arange(len(sorted_idx))[-TOP:], feature_importance[sorted_idx][-TOP:], align='center')
plt.yticks(np.arange(len(sorted_idx))[-TOP:], np.array(FEATURES)[sorted_idx][-TOP:])
plt.title(f'Feature Importance - Top {TOP}')
plt.show()

# CV Score for CatBoost
This is CV score for our CatBoost model.

In [ ]:
import sys
sys.path.append('../input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

oof.head()

In [ ]:
true.head()

In [ ]:
# TARGETSの各列を正規化
oof_data = oof[[0, 1, 2, 3, 4, 5]].values
oof_data = oof_data / oof_data.sum(axis=1,keepdims=True)
oof[[0, 1, 2, 3, 4, 5]] = oof_data

In [ ]:
oof.head()

In [ ]:
temp = pl.from_pandas(oof)

In [ ]:
cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for CatBoost =',cv)

In [ ]:
import sys
sys.path.append('../input/kaggle-kl-div')
from kaggle_kl_div import score

oof = pd.DataFrame(all_oof.copy())
oof['id'] = np.arange(len(oof))

true = pd.DataFrame(all_true.copy())
true['id'] = np.arange(len(true))

cv = score(solution=true, submission=oof, row_id_column_name='id')
print('CV Score KL-Div for CatBoost =',cv)

# CV Score KL-Divの保存
with open(f'models/CV_Score_v{VER}.txt', 'w') as f:
    f.write(str(cv) + "\n")